Summarize for us the goal of this project and how machine learning is useful in trying to accomplish it. As part of your answer, give some background on the dataset and how it can be used to answer the project question. Were there any outliers in the data when you got it, and how did you handle those?  [relevant rubric items: “data exploration”, “outlier investigation”]

### Response for the questions of Enron dataset project for intro to Machine Learning (Udacity)

#### @author: Hung-Tao Chou, hungtao@gmail.com

#### Date: Oct 22, 2016

For the details of workflow of this project, please refer to: 

http://nbviewer.jupyter.org/github/nanoman08/ud120-projects/blob/master/final_project/enron_final_project_HT.ipynb

### Q1: Summarize for us the goal of this project and how machine learning is useful in trying to accomplish it. As part of your answer, give some background on the dataset and how it can be used to answer the project question. Were there any outliers in the data when you got it, and how did you handle those? [relevant rubric items: “data exploration”, “outlier investigation”]

The goal of this project is to identiy the person of interest in the Enron datset: one the biggest Corporate Fraud. In this project, we are going to investigate the fraud case of the Enron, which is one the largest companies in the United States in 2000, and which by 2002 had collapsed into bankruptcy due to widespread corporate fraud. In the resulting Federal investigation, there was a significant amount of typically confidential information availabe to the public, including tens of thousands of emails and detailed financial data for top executives. For this project, we are going to try to indentify the "person of interest" (poi) from the people (146 data points total) involved by using the machine learning technique. In the Enron dataset, 21 features (with 'poi' as one of the feature, so actually 20 features total as the input variable) are provided and other information such as the emails of each individual people are provided.

For this project, we use the very popular package - scikit-learn package to create our models. We first load the dataset as a dictionary (my_dataset). 

#### 1st outlier point: when using the featureformat function, the name of 'LOCKHART EUGENE E' is filtered out as the data point has no value of all features at all. 

At this stage, based on what the features represent, we have decided to generate three new features first:
new_feature = ['norm_from_poi_to_this_person', 'norm_shared_receipt_with_poi', 'norm_from_this_person_to_poi' ]
These are the ratio of from_poi_to_this_person vs. from_message, ratio of shared_receipt_with_poi vs. to_message, ratio of from_this_person_to_poi vs. to_message.

And in the original 20 features/variables, one of them is "email" which will not be used as input for classification. So overall we have 19 original + 3 new = 22 total features. 

#### 2nd outlifer point: When plotting the binning plot of each features of "poi" and "non-poi" together, it is clear seen there is an outlier of a lot of features which all have much larger value. And they all link the 'name' of "TOTAL", which is the sum of all the other people's value. So this "TOTAL" data point is filtered out also



### Q2: What features did you end up using in your POI identifier, and what selection process did you use to pick them? Did you have to do any scaling? Why or why not? As part of the assignment, you should attempt to engineer your own feature that does not come ready-made in the dataset -- explain what feature you tried to make, and the rationale behind it. (You do not necessarily have to use it in the final analysis, only engineer and test it.) In your feature selection step, if you used an algorithm like a decision tree, please also give the feature importances of the features that you use, and if you used an automated feature selection function like SelectKBest, please report the feature scores and reasons for your choice of parameter values.  [relevant rubric items: “create new features”, “properly scale features”, “intelligently select feature”]



I used the previous 22 features to start first. The new features I added is what decribed previously.
new_feature = ['norm_from_poi_to_this_person', 'norm_shared_receipt_with_poi', 'norm_from_this_person_to_poi' ]

To try out different classifier, I used 4 difference classifier: logistic, decision tree, random forrest and Adaptive boosting. 
I didnt do any scaling since for all thse 4 classifier, there is no need to do scaling (scaling would be needed for distance-based such as SVC or KNN). ##### One thing to note is due the unbalanced numbers between 'poi' (only 18) and 'non-poi (>100), I specified the weight = 'balanced' in all the classifier, the classifier improves a lot after this modification.


Then I use the RFECV method to select out the features: Feature ranking with recursive feature elimination and cross-validated selection of the best number of features, and I use F1 as the scoring to select the best number of features.

http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html#sklearn.feature_selection.RFECV


### Q3: What algorithm did you end up using? What other one(s) did you try? How did model performance differ between algorithms?  [relevant rubric item: “pick an algorithm”]

The F1 score vs. features of the 4 different classfier are all plotted out in my details ipython notebook (link on the top), the Adaptive boosting has the highest F1 score. It's interesting to note that logistic regression has very good recall score > 0.8 even though the F1 score is low (also low precision score).  
Instead of picking one model out, I decided to fine tune all the models to gain more insights.  

### Q4: What does it mean to tune the parameters of an algorithm, and what can happen if you don’t do this well?  How did you tune the parameters of your particular algorithm? (Some algorithms do not have parameters that you need to tune -- if this is the case for the one you picked, identify and briefly explain how you would have done it for the model that was not your final choice or a different model that does utilize parameter tuning, e.g. a decision tree classifier).  [relevant rubric item: “tune the algorithm”]

Tuning the parameters of an algorithm means finding the parameters of an algorithm that provides the best score, and if this is not done well, it is possible the algorithm is tuned and trapped into a local optimum but not the best optimum.  

Right after the RFECV selected the features, I use the GridSearchCV to further fine tune the models. For each model, I specify a range of parameters for the GridSearchCV, using the Adaptive boosting as an example. 

param_grid = {
    "n_estimators": range(20,50,300),
    "learning_rate": [0.1, 1, 10, 100],
    "algorithm": ['SAMME', 'SAMME.R']
    }    
    
The above is the parameter grid Adaptive boosting search through and the CV in the GridSearchCV stands for cross-validation, which I will cover in the Q5. 

### Q5: What is validation, and what’s a classic mistake you can make if you do it wrong? How did you validate your analysis?  [relevant rubric item: “validation strategy”]

The validation is to validate if the model will work for future dataset. If the model is to built on the whole dataset provided, then it is not possible to test how the model will work for futre dataset, and if our model is over-fit on the provided dataset, it will be able to predict the future dataset very well. In order to do the validation, we can seperate the whole dataset into two parts: Train and Test. The train dataset is used to train and build the model. And the test dataset is used to test how good the trained model is. 
In the GridSearchCV method, it already has implement the cross validation technique which split the data into train/test dataset. 
http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html

Using the Adaptive boosting as an example:

clf_AdaB_searchCV = GridSearchCV(clf, param_grid, scoring = 'f1', cv=10)

The cv = 10 means the a Stratified 10 fold is used for cross validation. Scoring = 'f1' means the testing score is 'f1'.
After searching through the grid and uses the cross-validation. The optimized Adaptive boosting model is:

AdaBoostClassifier(algorithm='SAMME',
          base_estimator=DecisionTreeClassifier(class_weight='balanced', criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=11,
            splitter='best'),
          learning_rate=0.1, n_estimators=20, random_state=25) AdaBoostClassifier(algorithm='SAMME',
          base_estimator=DecisionTreeClassifier(class_weight='balanced', criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=11,
            splitter='best'),
          learning_rate=0.1, n_estimators=20, random_state=25)




### Q6: Give at least 2 evaluation metrics and your average performance for each of them.  Explain an interpretation of your metrics that says something human-understandable about your algorithm’s performance. [relevant rubric item: “usage of evaluation metrics”]


##### Summary of the 4 models:

Classifier | Features used|  Accuracy | Precision  | Recall   | F1 score |
--- | --- | --- | --- | --- | --- | --- 
*Logisitc* |8 | 0.797 | 0.354 | 0.634 |0.454
*Decision Tree* | 5 | 0.775 | 0.340 | 0.732 |0.464
*Random Forrest* | 6 | 0.817 | 0.385 | 0.619 |0.474 
*Ada boosting* | 7 | 0.867 | 0.5 | 0.517 |0.508 

The Adaptive boosting model provides the best F1 score among the 4 models, with ~ 0.5 of Precsion and Recall, and therefore F1 is ~ 0.5 also. This means the model can catch about 50% of the total 'poi'. In the mean time, the accuracy is also about 50%, about half of the prediction is wrong. This is much better than other three models which the accuracy is all below 40%.  

It's interesting to note although the rest 3 have lower F1 score, they all have a much higher Recall score. The Decision Tree classifier has the highest Recall score: 0.732. 
In the Fraud case like this, even though the precision is low for the Decision Tree model, but the higher recall score might suit better for the initial screening for poi since it is better to catch as much poi as possible and later can dig in more to filter out the wrong identified ones.

Just for future fun, it will be interesting to build more features out of the content of the emails, and see if specifically this can help to imporve the accuracy of the Decision Tree algorithm, while still keeping the high Recall score. 


